In [3]:
# import the necessary packages
import imutils
def pyramid(image, scale=1.5, minSize=(30, 30)):
	# yield the original image
	yield image
	# keep looping over the pyramid
	while True:
		# compute the new dimensions of the image and resize it
		w = int(image.shape[1] / scale)
		image = imutils.resize(image, width=w)
		# if the resized image does not meet the supplied minimum
		# size, then stop constructing the pyramid
		if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
			break
		# yield the next image in the pyramid
		yield image
def sliding_window(image, stepSize, windowSize):
	# slide a window across the image
	for y in range(0, image.shape[0], stepSize):
		for x in range(0, image.shape[1], stepSize):
			# yield the current window
			yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [4]:
import cv2
import time

image = cv2.imread("frames/seq_000001.jpg")
(winW, winH) = (128, 128)

In [5]:
# loop over the image pyramid
for resized in pyramid(image, scale=1.5):
	# loop over the sliding window for each layer of the pyramid
	for (x, y, window) in sliding_window(resized, stepSize=32, windowSize=(winW, winH)):
		# if the window does not meet our desired window size, ignore it
		if window.shape[0] != winH or window.shape[1] != winW:
			continue
			
			
		# THIS IS WHERE YOU WOULD PROCESS YOUR WINDOW, SUCH AS APPLYING A
		# MACHINE LEARNING CLASSIFIER TO CLASSIFY THE CONTENTS OF THE
		# WINDOW
		# since we do not have a classifier, we'll just draw the window
		clone = resized.copy()
		cv2.rectangle(clone, (x, y), (x + winW, y + winH), (0, 255, 0), 2)
		cv2.imshow("Window", clone)
		cv2.waitKey(1)

In [1]:
import matplotlib.image as mpimg
import cv2
import numpy as np
import time
import random as rg
import math
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

file1 = "positive\Image73_person_8.jpg"
file2 = "positive\Image73_person_9.jpg"

img1 = cv2.imread(file1, 0)
print(img1.shape)
img1 = cv2.resize(img1, (64, 128))
print(img1.shape)

img2 = cv2.imread(file2, 0)
print(img2.shape)
img2 = cv2.resize(img2, (64, 128))
print(img2.shape)

R_train_images = np.array([img1, img2])
print(R_train_images.shape)

positive_path = "./positive"
negative_path = "./negative"

(58, 23)
(128, 64)
(97, 22)
(128, 64)
(2, 128, 64)


In [2]:
def CreateTrainSet(positive_path, negative_path, IMAGE_WIDTH, IMAGE_HEIGHT, Positive_Images=1200):
  # getting all file names from positive path
  positives = os.listdir(positive_path)
  positive_files = [os.path.join(positive_path, file_name) for file_name in positives if file_name.endswith('.jpg')]
  positive_files.sort()

  # getting all file names from negative path
  negatives = os.listdir(negative_path) 
  negative_files = [os.path.join(negative_path, file_name) for file_name in negatives if file_name.endswith('.jpg')]
  negative_files.sort()

  # creating train label np array for pos=0 and neg=1
  pos_labels = np.zeros(Positive_Images)
  neg_labels = np.ones(len(negative_files))
  train_labels = np.concatenate((pos_labels, neg_labels), axis=0).astype(int)
  
  # add positive images to train_image np array
  pos_images = np.zeros((Positive_Images, IMAGE_HEIGHT, IMAGE_WIDTH))
  for filename in positive_files[0: Positive_Images]:
    img = cv2.imread(filename, 0)
    img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    pos_images[positive_files.index(filename)] = img
  
  # add negative images to train_image np array
  neg_images = np.zeros((len(negative_files), IMAGE_HEIGHT, IMAGE_WIDTH))
  for filename in negative_files:
    img = cv2.imread(filename, 0)
    img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    neg_images[negative_files.index(filename)] = img
  
  train_images = np.zeros((len(positive_files)+len(negative_files), IMAGE_HEIGHT, IMAGE_WIDTH))
  train_images = np.concatenate((pos_images, neg_images), axis=0).astype(int)
  return train_images, train_labels

In [3]:
Positive_Images = 1200


print(f"Path exists: {os.path.isdir(positive_path) and os.path.isdir(negative_path)}")
IMAGE_WIDTH = 64
IMAGE_HEIGHT = 128
R_train_images, train_labels = CreateTrainSet(positive_path, negative_path, IMAGE_WIDTH, IMAGE_HEIGHT, Positive_Images)

print("train_images: ", R_train_images.shape)
print("train_labels: ", train_labels.shape)
print(R_train_images[0])
print(train_labels[0])
print(R_train_images[-1])
print(train_labels[-1])

Path exists: True
train_images:  (1480, 128, 64)
train_labels:  (1480,)
[[196 197 201 ... 124 119 117]
 [195 197 200 ... 125 118 115]
 [195 196 200 ... 126 116 111]
 ...
 [181 181 181 ... 182 181 181]
 [178 178 177 ... 184 184 184]
 [176 176 175 ... 186 186 186]]
0
[[198 198 197 ... 123 113 106]
 [196 196 196 ... 121 115 113]
 [194 194 194 ... 121 123 128]
 ...
 [247 247 246 ... 246 245 245]
 [250 250 248 ... 244 244 244]
 [247 247 246 ... 246 247 247]]
1


In [4]:
# returns HoG features, and orderd features
def HoG_features(images):
    cell_size = (8,8)
    block_size = (4,4)
    nbins = 4

    # all images have same shape
    img_size = images[0].shape

    # creating HoG object
    hog = cv2.HOGDescriptor(_winSize=(img_size[1] // cell_size[1] * cell_size[1],
                                    img_size[0] // cell_size[0] * cell_size[0]),
                            _blockSize=(block_size[1] * cell_size[1],
                                        block_size[0] * cell_size[0]),
                            _blockStride=(cell_size[1], cell_size[0]),
                            _cellSize=(cell_size[1], cell_size[0]),
                            _nbins=nbins)

    features = []
    for i in range(images.shape[0]):
        
        # Compute HoG features
        features.append(hog.compute((images[i]).astype(np.uint8)).reshape(1, -1))
    
    # Stack arrays in sequence vertically 
    features = np.vstack(features)
   
    return features

In [5]:
# getting HoG features
train_features = HoG_features(R_train_images)
# test_features = HoG_features(R_test_images, FEATURE_SIZE)

print("trained_features_reshaped: ", train_features.shape)
print("trained_features_reshaped[0]: ", train_features[0])
# print("test_features_reshaped: ", test_features.shape)
# print("test_features_reshaped[0]: ", test_features[0])

trained_features_reshaped:  (1480, 4160)
trained_features_reshaped[0]:  [0.03915166 0.0065741  0.00676362 ... 0.02321829 0.02239115 0.00087363]


In [19]:
def NonLinear_SVM(train_features, train_labels, gamma, C, random_state=None):
    # creating non-linear svc object, RBF kernel is default
    clf = svm.SVC(C=C, gamma=gamma, random_state=random_state)

    # fit and predict
    clf.fit(train_features, train_labels)
    return clf

def predict(clf, test_features, test_labels):
    predict = clf.predict(test_features)
    
    # using accruacy score from metrics lib and multiply 100 to get precentage
    accuracy = accuracy_score(test_labels, predict)*100
    return accuracy

In [7]:
k_fold = 5
pos_count = Positive_Images
neg_count = 280
pos_train_split = int(pos_count*4/k_fold)
neg_train_split = int(pos_count+neg_count*4/k_fold)

print(f"train_size: {pos_train_split+neg_train_split-pos_count}")
print(f"test_size: {pos_count-pos_train_split+neg_count-neg_train_split+pos_count}")

# splitting all pos and neg into 4/5 for train and 1/5 split for test
train_features_split = np.concatenate((train_features[: pos_train_split],  train_features[pos_count: neg_train_split]), axis=0)
train_labels_split = np.concatenate((train_labels[: pos_train_split], train_labels[pos_count: neg_train_split]), axis=0)

val_features_split = np.concatenate((train_features[pos_train_split:pos_count],  train_features[neg_train_split:]), axis=0)
val_labels_split = np.concatenate((train_labels[pos_train_split:pos_count],  train_labels[neg_train_split:]), axis=0)

print(f"train_split: {train_features_split.shape} and {train_labels_split.shape}")
print(f"val_split: {val_features_split.shape} and {val_labels_split.shape}")

train_size: 1184
test_size: 296
train_split: (1184, 4160) and (1184,)
val_split: (296, 4160) and (296,)


In [20]:
# Optimal SVM Classifer
gamma = "scale"
C = 10
Optimal_Clf = NonLinear_SVM(train_features_split, train_labels_split, gamma, C)
accuracy = predict(clf, val_features_split, val_labels_split)
print(f"Gamma: {gamma}, C: {C},  Accuracy: {round(accuracy, 2)}%")

Gamma: scale, C: 10,  Accuracy: 96.62%


In [10]:
MIN_ACCURACY = 50
GammaList = ['auto', 'scale']
C_List = [0.01, 0.1, 1, 10, 100, 1000]
Best_SVM = {"gamma":None, "C":None, "accuracy":0}
for gamma in GammaList:
    for C in C_List:
        clf = NonLinear_SVM(train_features_split, train_labels_split, gamma, C)
        accuracy = predict(clf, val_features_split, val_labels_split)
        if round(accuracy, 2) > MIN_ACCURACY:
            print(f"Gamma: {gamma}, C: {C},  Accuracy: {round(accuracy, 2)}%")
        if round(accuracy, 2) > Best_SVM["accuracy"]:
            Best_SVM["gamma"] = gamma
            Best_SVM["C"] = C
            Best_SVM["accuracy"] = round(accuracy, 2)
print("Best parameters: ", Best_SVM)

Gamma: auto, C: 0.01,  Accuracy: 81.08%
Gamma: auto, C: 0.1,  Accuracy: 81.08%
Gamma: auto, C: 1,  Accuracy: 81.08%
Gamma: auto, C: 10,  Accuracy: 87.16%
Gamma: auto, C: 100,  Accuracy: 95.27%
Gamma: auto, C: 1000,  Accuracy: 95.95%
Gamma: scale, C: 0.01,  Accuracy: 81.08%
Gamma: scale, C: 0.1,  Accuracy: 83.45%
Gamma: scale, C: 1,  Accuracy: 95.95%
Gamma: scale, C: 10,  Accuracy: 96.62%
Gamma: scale, C: 100,  Accuracy: 96.62%
Gamma: scale, C: 1000,  Accuracy: 96.62%
Best parameters:  {'gamma': 'scale', 'C': 10, 'accuracy': 96.62}


In [13]:
def k_fold_SVC(train_features, train_labels, train_index, val_index, k_folds):
    total_accuracy = 0
    for i in range(k_folds):
        x_train, x_val = train_features[train_index], train_features[val_index]
        y_train, y_val = train_labels[train_index], train_labels[val_index]
        clf = NonLinear_SVM(x_train, y_train, gamma, C)
        total_accuracy += predict(clf, x_val, y_val)
    avg_accuracy = total_accuracy/k_folds
    return avg_accuracy

In [14]:
# 5 fold cross validation dataset
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True)
kf.get_n_splits(train_features)

MIN_ACCURACY = 50
GammaList = ['auto', 'scale']
C_List = [0.01, 0.1, 1, 10, 100, 1000]
Best_SVM = {"gamma":None, "C":None, "accuracy":0}
for gamma in GammaList:
    for C in C_List:
        start_time = time.time()
        for train_index, val_index in kf.split(train_features):
            accuracy = k_fold_SVC(train_features, train_labels, train_index, val_index, 5)
        time_taken = (time.time() - (start_time))/k_folds
        if round(accuracy, 2) > MIN_ACCURACY:
            print(f"Gamma: {gamma}, C: {C},  Accuracy: {round(accuracy, 2)}%, time taken to train/test: {round(time_taken, 2)}")
        if round(accuracy, 2) > Best_SVM["accuracy"]:
            Best_SVM["gamma"] = gamma
            Best_SVM["C"] = C
            Best_SVM["accuracy"] = round(accuracy, 2)
print("Best parameters: ", Best_SVM)

Gamma: auto, C: 0.01,  Accuracy: 88.18%, time taken to train/test: 35.54
Gamma: auto, C: 0.1,  Accuracy: 83.11%, time taken to train/test: 33.97
Gamma: auto, C: 1,  Accuracy: 81.08%, time taken to train/test: 30.3
Gamma: auto, C: 10,  Accuracy: 86.49%, time taken to train/test: 30.58
Gamma: auto, C: 100,  Accuracy: 94.93%, time taken to train/test: 22.84
Gamma: auto, C: 1000,  Accuracy: 95.95%, time taken to train/test: 17.95
Gamma: scale, C: 0.01,  Accuracy: 77.36%, time taken to train/test: 28.71
Gamma: scale, C: 0.1,  Accuracy: 82.09%, time taken to train/test: 32.66
Gamma: scale, C: 1,  Accuracy: 96.28%, time taken to train/test: 27.2
Gamma: scale, C: 10,  Accuracy: 97.97%, time taken to train/test: 29.54
Gamma: scale, C: 100,  Accuracy: 97.64%, time taken to train/test: 28.62
Gamma: scale, C: 1000,  Accuracy: 97.64%, time taken to train/test: 26.75
Best parameters:  {'gamma': 'scale', 'C': 10, 'accuracy': 97.97}


In [ ]:
import csv
detectron_file = open(path + 'detectron_count.csv')
detectron_reader = csv.reader(detectron_file, delimiter=',')

accfile = open(path + 'accuracy_comparison.csv')
acc_writer = csv.reader(accfile, delimiter=',')
acc_writer.writerow(["id", "accuracy"])

svm_file = open(path + 'svm_submission.csv', 'w+', newline='')
svm_writer = csv.writer(svm_file)
svm_writer.writerow(["id", "accuracy"])

line_count = 0
for row in detectron_reader:
        line_count += 1
    else:
        writer.writerow([row[0], row[1]])
        line_count += 1
print(f'Processed {line_count} lines.')

cfile.close
accfile.close
svm_file.close